In [1]:
import json
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import numpy as np

/home/suigpt_rag/miniconda3/envs/new_CG/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# 1. import testing Question dataset [Q]
with open("../math_qa_all_v1.json", "r") as qf:
    data = json.load(qf)
    page_content = [q['content'] for q in data]
    page_number = [q['page'] for q in data]
    questions: list = [q['Q'] for q in data]
    answers: list = [q['A'] for q in data]

FileNotFoundError: [Errno 2] No such file or directory: '../math_qa_all.json_v1'

In [6]:
model_name = 'thenlper/gte-large'
model = SentenceTransformer(model_name)

In [8]:
page_embeddings: np.ndarray = model.encode(page_content) 
questions_embeddings: np.ndarray  = model.encode(questions)

In [10]:
for i in range(len(data)):
    data[i]['page_embedding'] = page_embeddings[i]
    data[i]['question_embedding'] = questions_embeddings[i]

In [12]:
# question_retrived_page_rank = []
for i in range(len(data)):
    similarity = [float(cos_sim(data[i]['question_embedding'], page_embed)) for page_embed in page_embeddings]
    question_retrived_page_rank = [{k: v} for k, v in zip(page_number, similarity)]
    data[i]['question_retrived_page_rank'] = question_retrived_page_rank

In [54]:
for i in range(len(data)):
    data[i]['question_retrived_page_rank'] = sorted(data[i]['question_retrived_page_rank'], key = lambda d: list(d.values())[0], reverse=True)

In [59]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0

page_distances = [] # the distances of the correct page and retrieved page
scores = [] # the score of the corresponding page and question
correct_rank = [] # the index of the correct page in the retrieved results.

for i in range(len(data)):
    page_number = data[i]['page']
    retrived_page_rank = [list(data[i]['question_retrived_page_rank'][d].keys())[0] for d in range(len(data[i]['question_retrived_page_rank']))]
    correct_page_rank = retrived_page_rank.index(page_number)
    correct_rank.append(correct_page_rank)
    page_distances.append(abs(page_number - retrived_page_rank[0]))
    scores.append(cos_sim(data[i]['question_embedding'], data[i]['page_embedding']))
    
    if page_number in retrived_page_rank[:1]: top1 += 1
    if page_number in retrived_page_rank[:3]: top3 += 1
    if page_number in retrived_page_rank[:5]: top5 += 1
    if page_number in retrived_page_rank[:10]: top10 += 1
    if page_number in retrived_page_rank[:20]: top20 += 1
    if page_number in retrived_page_rank[:50]: top50 += 1
    if page_number in retrived_page_rank[:100]: top100 += 1   

In [60]:
print(top1)
print(top3)
print(top5)
print(top10)
print(top20)
print(top50)
print(top100)


228
349
394
435
471
489
493


In [70]:
print("Avg score: " + str(float(sum(scores) / len(scores))))
print("Avg rank: " + str(sum(correct_rank) / len(correct_rank)))
print("Avg page distance: " + str(sum(page_distances)/len(page_distances)))

Avg score: 0.8884187936782837
Avg rank: 6.3453815261044175
Avg page distance: 24.204819277108435
